In [1]:
from elasticsearch import  Elasticsearch

In [2]:
#connect to elastic search
es_host = 'https://192.168.88.16:31419'
api_key = 'your api key'
client = Elasticsearch(es_host, verify_certs=False,api_key=api_key)
print(client.info())
print(client.ping())

{'name': 'quickstart-es-default-0', 'cluster_name': 'quickstart', 'cluster_uuid': '2nnOvh7USDO7LHKmWliVIg', 'version': {'number': '8.14.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'd55f984299e0e88dee72ebd8255f7ff130859ad0', 'build_date': '2024-07-07T22:04:49.882652950Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}
True


d:\amenities recommendations\.venv\lib\site-packages\elasticsearch\_sync\client\__init__.py:400: SecurityWarning: Connecting to 'https://192.168.88.16:31419' using TLS with verify_certs=False is insecure
  _transport = transport_class(
d:\amenities recommendations\.venv\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.88.16'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\amenities recommendations\.venv\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.88.16'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [3]:
#load amenities from the json file
import pandas as pd

amenities = pd.read_json('amenities.json')
amenities.head()
amenities.info()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   idAmenity    22 non-null     int64 
 1   description  22 non-null     object
 2   nameAmenity  22 non-null     object
 3   categories   22 non-null     object
 4   price        22 non-null     object
dtypes: int64(1), object(4)
memory usage: 1008.0+ bytes


In [4]:
#import the sentence transformer model to encode the amenities(embeddings)
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

d:\amenities recommendations\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


d:\amenities recommendations\.venv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\CharbelMazloum\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [5]:
amenities['description_embeddings'] = amenities['description'].apply(lambda x: model.encode(x))
amenities['nameAmenity_embeddings'] = amenities['nameAmenity'].apply(lambda x: model.encode(x))
amenities['categories_embeddings'] = amenities['categories'].apply(lambda x: model.encode(x))
amenities['price_embeddings'] = amenities['price'].apply(lambda x: model.encode(x))
print(amenities.columns)
print(amenities.head())

Index(['idAmenity', 'description', 'nameAmenity', 'categories', 'price',
       'description_embeddings', 'nameAmenity_embeddings',
       'categories_embeddings', 'price_embeddings'],
      dtype='object')
   idAmenity                                        description  \
0          1  An easy-to-use bedside alarm clock with large,...   
1          2  A comprehensive cleaning service for your stud...   
2          3  A thorough cleaning of the suite designed to m...   
3          4  A convenient scooter rental service that allow...   
4          5  High-speed WiFi connectivity available through...   

       nameAmenity                             categories    price  \
0      Alarm Clock            Room Amenities, Convenience  No fees   
1  Cleaning Studio    Cleaning Services, Room Maintenance   10 USD   
2   Cleaning Suite    Cleaning Services, Room Maintenance   50 USD   
3   Scooter Rental     Transportation, Leisure Activities  100 USD   
4             WiFi  Internet Services, B

In [7]:

indexMapping ={
    "properties":{
        "idAmenity":{"type":"long"},
        "nameAmenity": {"type":"text"},
        "description": {"type":"text"},
        "categories": {"type":"text"},
        "price": {"type":"text"},
        "description_embeddings": {"type":"dense_vector","dims":768, "index":True, "similarity":"cosine"},
        "nameAmenity_embeddings": {"type":"dense_vector","dims":768, "index":True, "similarity":"cosine"},
        "categories_embeddings": {"type":"dense_vector","dims":768, "index":True, "similarity":"cosine"},
        "price_embeddings": {"type":"dense_vector","dims":768, "index":True, "similarity":"cosine"}
    }
}

In [8]:

record_list = amenities.to_dict(orient='records')
#if exsists delete the index
if client.indices.exists(index='amenities_data'):
    client.indices.delete(index='amenities_data')
#create index in es
client.indices.create(index='amenities_data', mappings=indexMapping)
client.indices.get(index='amenities_data').meta

d:\amenities recommendations\.venv\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.88.16'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\amenities recommendations\.venv\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.88.16'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\amenities recommendations\.venv\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.88.16'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ApiResponseMeta(status=200, http_version='1.1', headers={'X-elastic-product': 'Elasticsearch', 'content-type': 'application/vnd.elasticsearch+json;compatible-with=8', 'Transfer-Encoding': 'chunked'}, duration=0.0028972625732421875, node=NodeConfig(scheme='https', host='192.168.88.16', port=31419, path_prefix='', headers={'user-agent': 'elasticsearch-py/8.15.1 (Python/3.10.9; elastic-transport/8.15.1)'}, connections_per_node=10, request_timeout=10.0, http_compress=False, verify_certs=False, ca_certs=None, client_cert=None, client_key=None, ssl_assert_hostname=None, ssl_assert_fingerprint=None, ssl_version=None, ssl_context=None, ssl_show_warn=True, _extras={}))

In [10]:
#index the data in elastic search
for record in record_list:
    try:
        client.index(index='amenities_data', document=record ,id=record['idAmenity'])
    except Exception as e:
        print(str(e))

d:\amenities recommendations\.venv\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.88.16'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\amenities recommendations\.venv\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.88.16'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\amenities recommendations\.venv\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.88.16'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\amenities recommendations\.venv

In [9]:
client.count(index='amenities_data')['count']

d:\amenities recommendations\.venv\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.88.16'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


0

In [ ]:
from spellchecker import SpellChecker

spell_checker = SpellChecker()
correction = spell_checker.correction(' worlld')
# print(correction)

world


In [ ]:
# client Side impl.


arrayInput =['buger ', 'king ', 'resturant ']
concatArrayInput = ' '.join(arrayInput)
correted_input = concatArrayInput + ' '
for word in correted_input.split():
    correction = spell_checker.correction(word)
    correted_input += correction + ' '
print(correted_input)

vectorized_input = model.encode(correted_input)

query = {
    'field':'nameAmenity_embeddings',
    'query_vector': vectorized_input,
    'k':5,
    'num_candidates':22


}
results = client.knn_search(index='amenities_data', knn=query, source=['nameAmenity'])
results = results._body['hits']['hits']
for res in results:
    print(res['_source']['nameAmenity'])
0

buger  king  resturant  burger king restaurant 
Breakfast Buffet
Private Chef Service
Room Service
Luxury Car Rental
Cleaning Suite


d:\amenities recommendations\.venv\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.88.16'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\CharbelMazloum\AppData\Local\Temp\ipykernel_1096\742046430.py:22: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  results = client.knn_search(index='amenities_data', knn=query, source=['nameAmenity'])
